Source: https://www.kaggle.com/code/san2deep/predict-discount-xgboost

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from math import sqrt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn import linear_model
from sklearn.metrics import r2_score
import plotly.express as px
from sklearn.tree import DecisionTreeRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error,r2_score
from sklearn.model_selection import cross_val_score

In [ ]:
train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')

In [ ]:
train.head()

,id,title,Rating,maincateg,platform,price1,actprice1,Offer %,norating1,noreviews1,star_5f,star_4f,star_3f,star_2f,star_1f,fulfilled1
0,16695,Fashionable & Comfortable Bellies For Women (...,3.9,Women,Flipkart,698,999,30.13%,38.0,7.0,17.0,9.0,6.0,3,3,0
1,5120,Combo Pack of 4 Casual Shoes Sneakers For Men ...,3.8,Men,Flipkart,999,1999,50.03%,531.0,69.0,264.0,92.0,73.0,29,73,1
2,18391,Cilia Mode Leo Sneakers For Women (White),4.4,Women,Flipkart,2749,4999,45.01%,17.0,4.0,11.0,3.0,2.0,1,0,1
3,495,Men Black Sports Sandal,4.2,Men,Flipkart,518,724,15.85%,46413.0,6229.0,1045.0,12416.0,5352.0,701,4595,1
4,16408,Men Green Sports Sandal,3.9,Men,Flipkart,1379,2299,40.02%,77.0,3.0,35.0,21.0,7.0,7,7,1


# EDA

In [ ]:
train.shape

(15730, 16)

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15730 entries, 0 to 15729
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          15730 non-null  int64  
 1   title       15730 non-null  object 
 2   Rating      15730 non-null  float64
 3   maincateg   15204 non-null  object 
 4   platform    15730 non-null  object 
 5   price1      15730 non-null  int64  
 6   actprice1   15730 non-null  int64  
 7   Offer %     15730 non-null  object 
 8   norating1   15052 non-null  float64
 9   noreviews1  15152 non-null  float64
 10  star_5f     15142 non-null  float64
 11  star_4f     15191 non-null  float64
 12  star_3f     15499 non-null  float64
 13  star_2f     15730 non-null  int64  
 14  star_1f     15730 non-null  int64  
 15  fulfilled1  15730 non-null  int64  
dtypes: float64(6), int64(6), object(4)
memory usage: 1.9+ MB


In [ ]:
train.describe()

,id,Rating,price1,actprice1,norating1,noreviews1,star_5f,star_4f,star_3f,star_2f,star_1f,fulfilled1
count,15730.000000,15730.000000,15730.000000,15730.000000,15052.000000,15152.000000,15142.000000,15191.000000,15499.000000,15730.000000,15730.000000,15730.000000
mean,10479.541577,4.012873,688.070693,1369.286777,3057.660776,423.976307,1585.239466,655.923310,357.260662,155.085188,275.500572,0.601526
std,6080.166276,0.298440,649.409586,1240.900227,11846.965689,1768.230384,6177.476241,2855.735531,1402.246610,558.650254,958.589075,0.489600
min,3.000000,0.000000,69.000000,42.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5212.000000,3.900000,349.000000,699.000000,63.000000,9.000000,30.000000,12.000000,7.000000,3.000000,6.000000,0.000000
50%,10458.500000,4.000000,474.000000,999.000000,308.000000,44.000000,150.000000,60.000000,34.000000,17.000000,30.000000,1.000000
75%,15766.750000,4.200000,699.000000,1299.000000,1526.000000,215.000000,788.000000,300.000000,172.000000,77.000000,140.000000,1.000000
max,20973.000000,5.000000,5998.000000,13499.000000,289973.000000,45448.000000,151193.000000,74037.000000,34978.000000,11705.000000,18060.000000,1.000000


In [ ]:
train['Offer %']=train['Offer %'].str[0:4]

In [ ]:
train['Offer %']=train['Offer %'].astype('float')

In [ ]:
train.loc[train['norating1'].isna()]

,id,title,Rating,maincateg,platform,price1,actprice1,Offer %,norating1,noreviews1,star_5f,star_4f,star_3f,star_2f,star_1f,fulfilled1
19,12336,Women Beige Heels Sandal,4.0,NaN,Flipkart,499,999,50.0,NaN,NaN,28.0,9.0,10.0,2,5,0
63,6597,Slides,3.9,Men,Flipkart,300,1500,80.0,NaN,NaN,NaN,NaN,10.0,5,9,1
68,5575,Women Copper Flats Sandal,4.1,NaN,Flipkart,349,999,65.0,NaN,27.0,NaN,NaN,22.0,5,15,1
91,4262,Slippers,3.6,NaN,Flipkart,213,249,14.4,NaN,NaN,NaN,NaN,466.0,281,476,1
116,2767,Ace Slip-On Running Shoes For Men (Black),4.3,NaN,Flipkart,2999,4999,40.0,NaN,NaN,46.0,27.0,5.0,1,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15583,11224,Jutis For Men (Black),3.7,NaN,Flipkart,494,499,1.0,NaN,NaN,22.0,NaN,4.0,4,8,0
15606,19633,Men Blue Clogs Sandal,3.8,Men,Flipkart,292,1249,76.6,NaN,NaN,NaN,123.0,81.0,29,66,1
15674,7209,Zod Runner V3 Wn s IDP Running Shoes For Women...,4.3,NaN,Flipkart,1699,3999,57.5,NaN,NaN,NaN,NaN,268.0,102,140,1
15709,14826,Enzo Beta Wn's Basketball Shoes For Women (Br...,3.9,NaN,Flipkart,1701,5499,69.0,NaN,NaN,NaN,NaN,6.0,2,10,1


In [ ]:
# There are null values in the norating1 and noreview1 columns
train['norating1']=train['norating1'].fillna(train['norating1'].sum())

In [ ]:
train['noreviews1']=train['noreviews1'].fillna(train['noreviews1'].sum())

In [ ]:
total_cus=train['norating1']+train['star_5f']+train['star_4f']+train['star_3f']+train['star_2f']+train['star_1f']

In [ ]:
train['total_cus']=total_cus

In [ ]:
train=train.drop(['star_5f','star_4f','star_3f','star_2f','star_1f'],axis=1)
train.shape

(15730, 12)

In [ ]:
train=train.drop(['norating1','noreviews1'],axis=1)

In [ ]:
# Mapping data string kategori menjadi label kelas
train['platform']=train['platform'].map({'Flipkart':1,'Amazon':0})

In [ ]:
train

,id,title,Rating,maincateg,platform,price1,actprice1,Offer %,fulfilled1,total_cus
0,16695,Fashionable & Comfortable Bellies For Women (...,3.9,Women,1,698,999,30.1,0,76.0
1,5120,Combo Pack of 4 Casual Shoes Sneakers For Men ...,3.8,Men,1,999,1999,50.0,1,1062.0
2,18391,Cilia Mode Leo Sneakers For Women (White),4.4,Women,1,2749,4999,45.0,1,34.0
3,495,Men Black Sports Sandal,4.2,Men,1,518,724,15.8,1,70522.0
4,16408,Men Green Sports Sandal,3.9,Men,1,1379,2299,40.0,1,154.0
...,...,...,...,...,...,...,...,...,...,...
15725,16554,Women Maroon Heels Sandal,4.3,Women,1,567,1199,52.7,0,1614.0
15726,19600,Women Multicolor Flats Sandal,3.9,Women,1,499,998,50.0,1,492.0
15727,6733,Women Navy Flats Sandal,3.7,Women,1,329,499,34.0,1,NaN
15728,3889,Red Chief Men's Sneakers,3.9,Men,0,2249,4499,50.0,1,851.0


In [ ]:
train.loc[train['title'].str.upper().str.contains('WOMEN') & train['maincateg'].isnull()]

,id,title,Rating,maincateg,platform,price1,actprice1,Offer %,fulfilled1,total_cus
19,12336,Women Beige Heels Sandal,4.0,NaN,1,499,999,50.0,0,46023964.0
68,5575,Women Copper Flats Sandal,4.1,NaN,1,349,999,65.0,1,NaN
224,17357,Women Beige Bellies Sandal,4.4,NaN,1,474,999,52.5,0,NaN
226,15843,"Best Quality Boots Latest Fashion ,Trending Bo...",4.0,NaN,1,349,999,65.0,1,NaN
328,19480,Bellies For Women (Gold),3.9,NaN,1,379,1499,74.7,1,NaN
...,...,...,...,...,...,...,...,...,...,...
15013,18584,Perfect Stylish Girls High Ankel Boots For Wom...,4.2,NaN,1,449,999,55.0,0,NaN
15502,11192,Jutis For Women (Yellow),4.0,NaN,1,199,499,60.1,0,NaN
15570,9968,Women Multicolor Bellies Sandal,3.7,NaN,1,501,999,49.8,1,NaN
15674,7209,Zod Runner V3 Wn s IDP Running Shoes For Women...,4.3,NaN,1,1699,3999,57.5,1,NaN


In [ ]:
train.loc[train['title'].str.upper().str.contains('WOMEN') & train['maincateg'].isnull(), 'maincateg'] = 'Women'

In [ ]:
train[train['id']==12336]

,id,title,Rating,maincateg,platform,price1,actprice1,Offer %,fulfilled1,total_cus
19,12336,Women Beige Heels Sandal,4.0,Women,1,499,999,50.0,0,46023964.0


In [ ]:
train.loc[train['title'].str.upper().str.contains('MEN') & train['maincateg'].isnull(), 'maincateg'] = 'Men'

In [ ]:
train['maincateg'].isnull().sum()

57

In [ ]:
train['maincateg'].unique()

array(['Women', 'Men', nan], dtype=object)

In [ ]:
train.loc[train['maincateg'].isnull(), 'maincateg'] = 'Unisex'
print(train['maincateg'].isnull().sum())
print(train['maincateg'].unique())

0
['Women' 'Men' 'Unisex']


In [ ]:
train.head()

,id,title,Rating,maincateg,platform,price1,actprice1,Offer %,fulfilled1,total_cus
0,16695,Fashionable & Comfortable Bellies For Women (...,3.9,Women,1,698,999,30.1,0,76.0
1,5120,Combo Pack of 4 Casual Shoes Sneakers For Men ...,3.8,Men,1,999,1999,50.0,1,1062.0
2,18391,Cilia Mode Leo Sneakers For Women (White),4.4,Women,1,2749,4999,45.0,1,34.0
3,495,Men Black Sports Sandal,4.2,Men,1,518,724,15.8,1,70522.0
4,16408,Men Green Sports Sandal,3.9,Men,1,1379,2299,40.0,1,154.0


In [ ]:
# pd.get_dummies is to create a copy of the dataframe
# but if we include something in the columns,
# we want that column to be encoded into some new label columns
# the values are only 0 and 1 for each label column
train=pd.get_dummies(train,columns=['maincateg'])

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html

In [ ]:
train.head()

,id,title,Rating,platform,price1,actprice1,Offer %,fulfilled1,total_cus,maincateg_Men,maincateg_Unisex,maincateg_Women
0,16695,Fashionable & Comfortable Bellies For Women (...,3.9,1,698,999,30.1,0,76.0,0,0,1
1,5120,Combo Pack of 4 Casual Shoes Sneakers For Men ...,3.8,1,999,1999,50.0,1,1062.0,1,0,0
2,18391,Cilia Mode Leo Sneakers For Women (White),4.4,1,2749,4999,45.0,1,34.0,0,0,1
3,495,Men Black Sports Sandal,4.2,1,518,724,15.8,1,70522.0,1,0,0
4,16408,Men Green Sports Sandal,3.9,1,1379,2299,40.0,1,154.0,1,0,0



The collections package in Python provides a set of specialized container datatypes beyond the built-in data types like lists, tuples, and dictionaries. One of the classes provided by the collections module is Counter, which is specifically designed for counting hashable objects. It's essentially a dictionary subclass for counting hashable objects, where the keys are the objects being counted and the values are their respective counts.

The Counter class is particularly useful when you need to count the occurrences of elements in a collection (such as a list or a string) or to keep track of frequencies of different items. It simplifies the process of counting elements and provides several helpful methods for working with the counts.


In [ ]:
# 100 most frequent words in title column

from collections import Counter

arr = []
for i in train['title'].str.split(' '): # the word should be splitted based on spaces
    arr += i

c = Counter(arr)
c_list = c.most_common(100)
c_list

# words = []

# for wd in c_list:
#     words.append(wd[0])
# print(words)

[('For', 10241),
 ('Sandal', 4194),
 ('Shoes', 4163),
 ('Women', 3976),
 ('Running', 2024),
 ('Sneakers', 1806),
 ('Men', 1580),
 ('Black', 1499),
 ('Boots', 1471),
 ('Casual', 1454),
 ('Men\xa0\xa0(Black)', 1454),
 ('Bellies', 1093),
 ('for', 1080),
 ('Jutis', 1073),
 ('Flats', 1054),
 ('Heels', 1008),
 ('Women\xa0\xa0(Black)', 967),
 ('Flip', 894),
 ('Wedges', 875),
 ('Flops', 843),
 ('Stylish', 830),
 ('Walking', 775),
 ('Slip', 711),
 ('Slippers', 679),
 ('On', 663),
 ('Sports', 653),
 ('&', 639),
 ('Slides', 618),
 ('Casuals', 614),
 ('shoes', 593),
 ('Girls', 592),
 ('Women\xa0\xa0(Pink)', 578),
 ('Men\xa0\xa0(Brown)', 529),
 ('|', 505),
 ('Leather', 469),
 ('of', 458),
 ('Mojaris', 458),
 ('Pink', 455),
 ("Men's", 445),
 ('Women\xa0\xa0(Multicolor)', 442),
 ('Lace', 435),
 ('Brown', 435),
 ('Women\xa0\xa0(White)', 432),
 ('and', 430),
 ('Combo', 403),
 ('Grey', 378),
 ('Blue', 371),
 ('Pack', 361),
 ('Women\xa0\xa0(Grey)', 360),
 ('Men\xa0\xa0(Blue)', 356),
 ('High', 351),
 ('La

In [ ]:
words = []

for wd in c_list:
    words.append(wd[0])
print(words)

['For', 'Sandal', 'Shoes', 'Women', 'Running', 'Sneakers', 'Men', 'Black', 'Boots', 'Casual', 'Men\xa0\xa0(Black)', 'Bellies', 'for', 'Jutis', 'Flats', 'Heels', 'Women\xa0\xa0(Black)', 'Flip', 'Wedges', 'Flops', 'Stylish', 'Walking', 'Slip', 'Slippers', 'On', 'Sports', '&', 'Slides', 'Casuals', 'shoes', 'Girls', 'Women\xa0\xa0(Pink)', 'Men\xa0\xa0(Brown)', '|', 'Leather', 'of', 'Mojaris', 'Pink', "Men's", 'Women\xa0\xa0(Multicolor)', 'Lace', 'Brown', 'Women\xa0\xa0(White)', 'and', 'Combo', 'Grey', 'Blue', 'Pack', 'Women\xa0\xa0(Grey)', 'Men\xa0\xa0(Blue)', 'High', 'Latest', 'Loafers', ',', '2', 'Men\xa0\xa0(White)', 'Women\xa0\xa0(Blue)', 'Up', 'Beige', 'Comfortable', "Women's", 'White', 'Red', 'men', 'Men\xa0\xa0(Tan)', 'And', 'Ethnic', 'Men\xa0\xa0(Multicolor)', 'Multicolor', 'Wear', 'Formal', 'Tan', 'Synthetic', 'Gold', 'Canvas', 'Party', 'Men\xa0\xa0(Grey)', 'Perfect', 'Training', 'IDP', 'Gym', 'Partywear', 'Jutti', 'Women\xa0\xa0(Brown)', 'Grey)', 'Ankle', 'Shoe', 'New', 'Women\xa

In [ ]:
train['combo'] = train['title'].str.upper().str.contains('COMBO').astype(int)

In [ ]:
train

,id,title,Rating,platform,price1,actprice1,Offer %,fulfilled1,total_cus,maincateg_Men,maincateg_Unisex,maincateg_Women,combo
0,16695,Fashionable & Comfortable Bellies For Women (...,3.9,1,698,999,30.1,0,76.0,0,0,1,0
1,5120,Combo Pack of 4 Casual Shoes Sneakers For Men ...,3.8,1,999,1999,50.0,1,1062.0,1,0,0,1
2,18391,Cilia Mode Leo Sneakers For Women (White),4.4,1,2749,4999,45.0,1,34.0,0,0,1,0
3,495,Men Black Sports Sandal,4.2,1,518,724,15.8,1,70522.0,1,0,0,0
4,16408,Men Green Sports Sandal,3.9,1,1379,2299,40.0,1,154.0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15725,16554,Women Maroon Heels Sandal,4.3,1,567,1199,52.7,0,1614.0,0,0,1,0
15726,19600,Women Multicolor Flats Sandal,3.9,1,499,998,50.0,1,492.0,0,0,1,0
15727,6733,Women Navy Flats Sandal,3.7,1,329,499,34.0,1,NaN,0,0,1,0
15728,3889,Red Chief Men's Sneakers,3.9,0,2249,4499,50.0,1,851.0,1,0,0,0


In [ ]:
sp_cols = ['Running','Black', 'Boots', 'Casual', 'Bellies', 'Jutis', 'Flats', 'Heels', 'Wedges', 'Stylish', 'Walking', 'Slip', 'Sports', 'Girls', 'Leather', 'Mojaris', 'Pink', 'Lace', 'Brown', 'Grey', 'Blue', 'Pack', 'Loafers', 'Beige', 'White', 'Red', 'Ethnic', 'Multicolor', 'Formal', 'Tan', 'Synthetic', 'Gold', 'Canvas', 'Party', 'Training', 'IDP', 'Gym', 'Jutti', 'Ankle', 'New', 'Navy', 'Lightweight', 'Derby']

In [ ]:
train['sandal'] = train['title'].str.upper().str.contains('SANDAL').astype(int)
train['sneaker'] = train['title'].str.upper().str.contains('SNEAKER').astype(int)
train['shoe'] = train['title'].str.upper().str.contains('SHOE').astype(int)
train['chappal'] = (train['title'].str.upper().str.contains('SLIPPERS') | train['title'].str.upper().str.contains('SLIDES') | train['title'].str.upper().str.contains('FLIP FLOPS')).astype(int)

In [ ]:
train.head()

,id,title,Rating,platform,price1,actprice1,Offer %,fulfilled1,total_cus,maincateg_Men,maincateg_Unisex,maincateg_Women,combo,sandal,sneaker,shoe,chappal
0,16695,Fashionable & Comfortable Bellies For Women (...,3.9,1,698,999,30.1,0,76.0,0,0,1,0,0,0,0,0
1,5120,Combo Pack of 4 Casual Shoes Sneakers For Men ...,3.8,1,999,1999,50.0,1,1062.0,1,0,0,1,0,1,1,0
2,18391,Cilia Mode Leo Sneakers For Women (White),4.4,1,2749,4999,45.0,1,34.0,0,0,1,0,0,1,0,0
3,495,Men Black Sports Sandal,4.2,1,518,724,15.8,1,70522.0,1,0,0,0,1,0,0,0
4,16408,Men Green Sports Sandal,3.9,1,1379,2299,40.0,1,154.0,1,0,0,0,1,0,0,0


In [ ]:
for cl in sp_cols: # Proses pembuatan setiap kolom dari sp_cols di data train
    train[cl] = train['title'].str.upper().str.contains(cl.upper()).astype(int)

In [ ]:
train

,id,title,Rating,platform,price1,actprice1,Offer %,fulfilled1,total_cus,maincateg_Men,...,Party,Training,IDP,Gym,Jutti,Ankle,New,Navy,Lightweight,Derby
0,16695,Fashionable & Comfortable Bellies For Women (...,3.9,1,698,999,30.1,0,76.0,0,...,0,0,0,0,0,0,0,0,0,0
1,5120,Combo Pack of 4 Casual Shoes Sneakers For Men ...,3.8,1,999,1999,50.0,1,1062.0,1,...,0,0,0,0,0,0,0,0,0,0
2,18391,Cilia Mode Leo Sneakers For Women (White),4.4,1,2749,4999,45.0,1,34.0,0,...,0,0,0,0,0,0,0,0,0,0
3,495,Men Black Sports Sandal,4.2,1,518,724,15.8,1,70522.0,1,...,0,0,0,0,0,0,0,0,0,0
4,16408,Men Green Sports Sandal,3.9,1,1379,2299,40.0,1,154.0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15725,16554,Women Maroon Heels Sandal,4.3,1,567,1199,52.7,0,1614.0,0,...,0,0,0,0,0,0,0,0,0,0
15726,19600,Women Multicolor Flats Sandal,3.9,1,499,998,50.0,1,492.0,0,...,0,0,0,0,0,0,0,0,0,0
15727,6733,Women Navy Flats Sandal,3.7,1,329,499,34.0,1,NaN,0,...,0,0,0,0,0,0,0,1,0,0
15728,3889,Red Chief Men's Sneakers,3.9,0,2249,4499,50.0,1,851.0,1,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
train.drop('title', axis='columns', inplace=True)

In [ ]:
train[train['Derby'] ==1]

,id,Rating,platform,price1,actprice1,Offer %,fulfilled1,total_cus,maincateg_Men,maincateg_Unisex,...,Party,Training,IDP,Gym,Jutti,Ankle,New,Navy,Lightweight,Derby
87,180,3.9,1,458,1624,79.60,1,230.0,1,0,...,0,0,0,0,0,0,0,0,0,1
240,16858,3.8,1,396,799,50.40,0,66.0,1,0,...,0,0,0,0,0,0,0,0,0,1
306,18124,4.2,1,949,999,5.01,0,1276.0,1,0,...,0,0,0,0,0,0,0,0,0,1
430,5880,3.9,1,949,2999,68.30,1,766.0,1,0,...,0,0,0,0,0,0,0,0,0,1
491,12785,3.9,1,765,999,23.40,0,86.0,1,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15190,17286,4.3,1,1859,6199,70.00,1,132.0,1,0,...,0,0,0,0,0,0,0,0,0,1
15325,9871,4.0,1,3354,6599,49.10,0,10.0,0,0,...,0,0,0,0,0,1,0,0,0,1
15403,6857,4.0,1,741,2249,67.00,0,102.0,1,0,...,0,0,0,0,0,0,0,0,0,1
15422,11562,4.3,1,1599,6399,75.00,1,52.0,1,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
train = train[(['Rating','actprice1', 'price1', 'fulfilled1', 'sandal', 'sneaker', 'shoe', 'chappal', 'maincateg_Men','maincateg_Women', 'platform','combo']) + sp_cols]
train.head()

,Rating,actprice1,price1,fulfilled1,sandal,sneaker,shoe,chappal,maincateg_Men,maincateg_Women,...,Party,Training,IDP,Gym,Jutti,Ankle,New,Navy,Lightweight,Derby
0,3.9,999,698,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,3.8,1999,999,1,0,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,4.4,4999,2749,1,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,4.2,724,518,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,3.9,2299,1379,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
x=train.drop(['price1'],axis=1)
y=train['price1']

In [ ]:
x.shape,y.shape

((15730, 54), (15730,))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state = 2233)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((9438, 54), (6292, 54), (9438,), (6292,))

# Machine Learning Model

## Decision Tree Regressor

In [ ]:
dt=DecisionTreeRegressor()

In [ ]:
dt.fit(X_train, y_train)

DecisionTreeRegressor()

In [ ]:
prediction_train = dt.predict(X_train)
prediction_train

array([640. , 298. , 229. , ..., 631.5, 277. , 304.2])

In [ ]:
prediction = dt.predict(X_test)
prediction

array([1650.,  199., 1047., ...,  499.,  255.,  319.])

In [ ]:
# Create a dataframe containing prediction and actual values for the test set as a slight comparison
test_results = pd.DataFrame(data={'predictions':prediction, 'actuals':y_test})
test_results

,predictions,actuals
3069,1650.0,925
624,199.0,399
7607,1047.0,1047
5212,562.0,562
986,525.0,483
...,...,...
8681,424.0,399
5752,408.0,379
5770,499.0,374
14840,255.0,253


In [ ]:
score=cross_val_score(dt,X_train,y_train,cv=5).mean()
score

0.8133699731601769

In [ ]:
def RMSE(aktual,prediction):
    error = aktual-prediction
    rmse = sqrt(sum(error**2)/len(error))
    return rmse

print("RMSE pada data train :" , RMSE(y_train,prediction_train))
print("RMSE pada data test  :" , RMSE(y_test,prediction))

RMSE pada data train : 41.59200807037266
RMSE pada data test  : 280.99467678632556


##LightGBM Regressor

In [ ]:
lgb=LGBMRegressor()
score=cross_val_score(lgb,X_train,y_train).mean()
score

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003419 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 352
[LightGBM] [Info] Number of data points in the train set: 7550, number of used features: 54
[LightGBM] [Info] Start training from score 683.986358
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003101 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 352
[LightGBM] [Info] Number of data points in the train set: 7550, number of used features: 54
[LightGBM] [Info] Start training from score 691.491656
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003575 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info]

0.8581998586832208

In [ ]:
lgb.fit(X_train, y_train)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003965 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 359
[LightGBM] [Info] Number of data points in the train set: 9438, number of used features: 54
[LightGBM] [Info] Start training from score 685.135092


LGBMRegressor()

In [ ]:
prediction_train = lgb.predict(X_train)
prediction_train

array([553.57744621, 355.33056792, 347.04274101, ..., 505.85034288,
       424.45540337, 269.64947181])

In [ ]:
prediction = lgb.predict(X_test)
prediction

array([1354.75737467,  363.02296881, 1155.06962039, ...,  345.57499995,
        229.82902037,  288.78162459])

In [ ]:
# Create a dataframe containing prediction and actual values for the test set as a slight comparison
test_results = pd.DataFrame(data={'predictions':prediction, 'actuals':y_test})
test_results

,predictions,actuals
3069,1354.757375,925
624,363.022969,399
7607,1155.069620,1047
5212,583.818089,562
986,488.167245,483
...,...,...
8681,518.689045,399
5752,455.657410,379
5770,345.575000,374
14840,229.829020,253


In [ ]:
def RMSE(aktual,prediction):
    error = aktual-prediction
    rmse = sqrt(sum(error**2)/len(error))
    return rmse

print("RMSE pada data train :" , RMSE(y_train,prediction_train))
print("RMSE pada data test  :" , RMSE(y_test,prediction))

RMSE pada data train : 193.2945926717154
RMSE pada data test  : 227.97918729174464


## XGBoost Regressor

In [ ]:
xgb=XGBRegressor()
score=cross_val_score(xgb,X_train,y_train).mean()
score

0.8741578903631669

In [ ]:
xgb.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [ ]:
prediction_train = xgb.predict(X_train)
prediction_train

array([575.29517, 329.3642 , 229.18849, ..., 492.57556, 344.18243,
       266.34155], dtype=float32)

In [ ]:
prediction = xgb.predict(X_test)
prediction

array([1243.8595 ,  376.81076, 1239.7913 , ...,  361.4303 ,  236.15683,
        284.6407 ], dtype=float32)

In [ ]:
# Create a dataframe containing prediction and actual values for the test set as a slight comparison
test_results = pd.DataFrame(data={'predictions':prediction, 'actuals':y_test})
test_results

,predictions,actuals
3069,1243.859497,925
624,376.810760,399
7607,1239.791260,1047
5212,619.077209,562
986,458.860504,483
...,...,...
8681,513.327576,399
5752,383.649963,379
5770,361.430298,374
14840,236.156830,253


In [ ]:
def RMSE(aktual,prediction):
    error = aktual-prediction
    rmse = sqrt(sum(error**2)/len(error))
    return rmse

print("RMSE pada data train :" , RMSE(y_train,prediction_train))
print("RMSE pada data test  :" , RMSE(y_test,prediction))

RMSE pada data train : 125.46118544345481
RMSE pada data test  : 218.3702614940357


LightGBM memberi hasil dengan perbedaan RMSE yang paling kecil antara data train dan data test nya.

Side Note:

XGBoost and LightGBM (LGB) are both popular libraries for gradient boosting, and both offer implementations for regression tasks. They share similarities in terms of the underlying boosting methodology, but they also have some differences in terms of their features, performance, and usage. Here's a comparison between XGBRegressor (XGBoost Regressor) and LGBMRegressor (LightGBM Regressor):

1. Performance and Speed:
XGBoost: XGBoost is known for its high performance and efficiency. It uses a combination of parallel processing and optimized data structures to achieve fast training times. XGBoost can handle large datasets well.
LightGBM: LightGBM is designed for high efficiency and is particularly well-suited for large datasets. It uses a histogram-based approach to speed up training, making it one of the fastest gradient boosting implementations.


2. Handling Categorical Features:
XGBoost: XGBoost requires categorical features to be encoded as integers. It does not handle categorical features natively.
LightGBM: LightGBM can directly handle categorical features without the need for explicit encoding. This can save preprocessing time and potentially improve model performance.

3. Tree Growth Strategy:
XGBoost: XGBoost uses a depth-first tree growth strategy.
LightGBM: LightGBM uses a leaf-wise tree growth strategy. This strategy can lead to faster training times but may also result in more overfitting if not carefully controlled.

4. Memory Usage:
XGBoost: XGBoost uses more memory compared to LightGBM, which might be a consideration when dealing with large datasets.

5. Tuning Complexity:
XGBoost: XGBoost has more hyperparameters to tune, which can make hyperparameter optimization more complex.
LightGBM: LightGBM also has hyperparameters, but its default settings are often reasonable and can provide good results with less tuning.

6. Parallelization:
XGBoost: XGBoost has a multi-threading option for parallelizing tree construction.
LightGBM: LightGBM is designed to take advantage of parallel and GPU computation for both tree construction and histogram creation.

7. Flexibility:
XGBoost: XGBoost provides more options for regularization, which can help prevent overfitting.
LightGBM: LightGBM focuses on speed and efficiency and might not have as many advanced regularization options as XGBoost.


Both XGBoost and LightGBM are well-regarded and widely used in the machine learning community. The choice between the two often depends on factors such as dataset size, desired speed, handling of categorical features, and ease of use. It's a good idea to experiment with both libraries on your specific problem to see which one performs better and suits your needs.

<br>

The histogram-based approach used in LightGBM is one of the key innovations that contribute to its speed and efficiency in training gradient boosting models. This approach is specifically designed to reduce the time complexity of constructing decision trees during the boosting process.

Here's how the histogram-based approach in LightGBM works:

1. Data Preprocessing:
Before building the trees, LightGBM bins the continuous feature values into discrete bins (also known as buckets or histograms). This process is done to transform the data into a more compact representation, which enables faster computations.

2. Histogram Construction:
For each feature, LightGBM constructs histograms that represent the distribution of the feature's values in each bin for every data point. These histograms are used to efficiently find the best split points during tree construction. The histograms are built incrementally, allowing them to be updated efficiently as trees are grown.

3. Candidate Split Selection:
When constructing a tree node, instead of considering all possible split points based on unique feature values, LightGBM uses the histograms to consider only a few candidate split points. These candidate split points are chosen based on quantiles of the histogram. This significantly reduces the number of possible split points and speeds up the search for the best split.

4. Best Split Selection:
LightGBM evaluates the quality of each candidate split point using the accumulated histogram values for each bin. It calculates the improvement in a chosen objective function (e.g., mean squared error for regression) for each candidate split. This avoids the need to scan all data points to calculate the exact impurity reduction, as in traditional decision tree algorithms.

5. Tree Growth:
The process of selecting the best split point and growing the tree is repeated recursively for each node. At each level, the histograms and candidate split points are used to efficiently find the best splits. The process continues until a stopping criterion (such as tree depth) is met.

The histogram-based approach offers advantages in terms of reduced time complexity because it avoids the need to sort and traverse all data points to find optimal splits. Instead, it works with histograms, which are more memory-friendly and faster to compute.

It's important to note that while the histogram-based approach contributes to LightGBM's efficiency, it can also lead to a tendency for overfitting, especially when the leaf-wise growth strategy is used. To mitigate overfitting, LightGBM provides various hyperparameters for controlling tree growth, such as max_depth, min_child_samples, and regularization options.